In [1]:
import geemap
import ee

from ipyleaflet import TileLayer, basemaps
from ipyleaflet.velocity import Velocity

import xarray as xr
import datetime

In [2]:
from siphon.catalog import TDSCatalog

In [3]:
ee.Initialize()

In [4]:
# Observed date in UTC
noon_wib = datetime.datetime(2021, 7, 7, 6)

In [5]:
# Use wind data from GFS
BASE_URL = 'http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/catalog.xml'
BEST_GFS = BASE_URL + '?dataset=grib/NCEP/GFS/Global_0p25deg/Best'

best_gfs = TDSCatalog(BEST_GFS)
best_ds = best_gfs.datasets[0]
ncss = best_ds.subset()

In [6]:
# Variable names
u_wind = 'u-component_of_wind_height_above_ground'
v_wind = 'v-component_of_wind_height_above_ground'

# GFS Query
query = ncss.query()

query.lonlat_box(north = -5, south = -9, east = 115, west = 105)
query.time(noon_wib)
query.variables(u_wind)
query.variables(v_wind)

data = ncss.get_data(query)
ds = xr.open_dataset(xr.backends.NetCDF4DataStore(data)) \
    .sel(height_above_ground4 = 10.0) \
    .sel(time = noon_wib)

In [7]:
center = [-6.2, 106.5]
zoom = 9

# Create map
m = geemap.Map(center=center, 
               zoom=zoom, 
               basemap=basemaps.CartoDB.DarkMatter, 
               add_google_map=False)

m

Map(center=[-6.2, 106.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [8]:
wind = Velocity(
    data=ds, 
    zonal_speed=u_wind, meridional_speed=v_wind, 
    latitude_dimension='lat', longitude_dimension='lon', 
    velocity_scale=0.01, max_velocity=20, 
)
m.add_layer(wind)

In [9]:
co_nrti = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_CO') \
    .filterDate(noon_wib.date().isoformat(), 
                (noon_wib + datetime.timedelta(days = 1)).date().isoformat()) \
    .select('CO_column_number_density').mean()

band_viz = {
    'min': 0,
    'max': 0.05,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

m.addLayer(co_nrti, band_viz, 'Carbon Monoxide')

In [10]:
indo_power = ee.FeatureCollection('WRI/GPPD/power_plants') \
    .filter(ee.Filter.eq('country', 'IDN'))

In [11]:
m.addLayer(indo_power, {'color' : 'ff0000'}, 'Power')